In [12]:
import pandas as pd
import torch
import matplotlib.pyplot as plt

torch.manual_seed(0)

In [13]:
df = pd.read_csv('iris.data', header=None)
df = df.sample(frac=1) #shuffle

# add label indices column
mapping = {k: v for v, k in enumerate(df [4]. unique())}
df[5] = df[4].map(mapping)

# normalise data
alldata = torch.tensor(df.iloc[:, [0,1,2,3]].values, dtype=torch.float)
alldata = (alldata - alldata.mean(dim=0)) / alldata.var(dim=0)
# create datasets
targets_tr = torch.tensor(df.iloc[:100, 5].values, dtype=torch.long)
targets_va = torch.tensor(df.iloc[100:, 5].values, dtype=torch.long)
data_tr = alldata [:100]
data_va = alldata [100:]


# Task 2.1

In [14]:
def mlp(data, W1, W2, b1, b2):
    return torch.relu(data @ W1 + b1) @ W2 + b2

def train_mlp(data_tr: torch.Tensor, targets_tr: torch.Tensor, data_va: torch.Tensor, targets_va: torch.Tensor, num_epochs: int = 100, lr: int = 0.01):
    # initialise weights and biases
    W1 = torch.rand((4, 12), requires_grad=True)
    W2 = torch.rand((12, 3), requires_grad=True)
    b1 = torch.rand(1, requires_grad=True)
    b2 = torch.rand(1, requires_grad=True)
    loss_tr_iter = torch.empty(num_epochs)
    loss_va_iter = torch.empty(num_epochs)

    for epoch in range(num_epochs):
        logits = mlp(data_tr, W1, W2, b1, b2)
        loss = torch.nn.functional.cross_entropy(logits, targets_tr, reduction="sum")
        loss.backward()

        # update weights
        with torch.no_grad():
            W1 -= lr * W1.grad
            W2 -= lr * W2.grad
            b1 -= lr * b1.grad
            b2 -= lr * b2.grad

        for params in [W1, W2, b1, b2]:
            params.grad.zero_()

        loss_tr_iter[epoch] = loss
        loss_va_iter[epoch] = torch.nn.functional.cross_entropy(
            mlp(data_va, W1, W2, b1, b2),
            targets_va,
            reduction="sum"
        )

    return W1, W2, b1, b2, loss_tr_iter, loss_va_iter


In [15]:
def accuracy(data, targets, W1, W2, b1, b2):
    logits = mlp(data, W1, W2, b1, b2)
    preds = torch.argmax(logits, axis=1)
    acc = (targets == preds).sum() / preds.shape[0]
    return acc

In [16]:
train_accs = []
val_accs = []
loss_trs = []
loss_vas = []

for i in range(10):
    W1, W2, b1, b2, loss_tr_iter, loss_va_iter = train_mlp(data_tr, targets_tr, data_va, targets_va)
    loss_tr_iter = loss_tr_iter.detach().numpy()
    loss_va_iter = loss_va_iter.detach().numpy()

    acc_tr = accuracy(data_tr, targets_tr, W1, W2, b1, b2)
    acc_va = accuracy(data_va, targets_va, W1, W2, b1, b2)
    train_accs.append(acc_tr)
    val_accs.append(acc_va)
    loss_trs.append(loss_tr_iter)
    loss_vas.append(loss_va_iter)

In [17]:
train_accs

[tensor(0.9200),
 tensor(0.6600),
 tensor(0.9200),
 tensor(0.7100),
 tensor(0.9200),
 tensor(0.9200),
 tensor(0.8800),
 tensor(0.9500),
 tensor(0.8500),
 tensor(0.7700)]

In [18]:
val_accs

[tensor(0.8800),
 tensor(0.6600),
 tensor(0.8800),
 tensor(0.7000),
 tensor(0.8800),
 tensor(0.8800),
 tensor(0.8200),
 tensor(0.8800),
 tensor(0.9000),
 tensor(0.7200)]

In [19]:
# fig, ax = plt.subplots(3, 2, figsize=(7, 6), sharex=True, sharey=True)
# for i in range(len(loss_trs)):
#     j, k = i // 2, i % 2
#     ax[j, k].plot(loss_trs[i])
#     ax[j, k].plot(loss_vas[i])
#     if i == 1:
#         ax[j, k].legend(["train", "validation"])

# fig.add_subplot(111, frameon=False)
# plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
# plt.xlabel("Epoch")
# plt.ylabel("Loss")

In [20]:
# fig.savefig("report/Figures/loss_curves.pdf")

In [26]:
df_accs = pd.DataFrame({"Training Accuracy": train_accs, "Validation Accuracy": val_accs}, dtype=float)
df_accs

,Training Accuracy,Validation Accuracy
0,0.92,0.88
1,0.66,0.66
2,0.92,0.88
3,0.71,0.70
4,0.92,0.88
5,0.92,0.88
6,0.88,0.82
7,0.95,0.88
8,0.85,0.90
9,0.77,0.72


In [27]:
print(df_accs.to_latex(caption="Accuracies of repeated MLP training.", label="tab:accs"))

\begin{table}
\centering
\caption{Accuracies of repeated MLP training.}
\label{tab:accs}
\begin{tabular}{lrr}
\toprule
{} &  Training Accuracy &  Validation Accuracy \\
\midrule
0 &               0.92 &                 0.88 \\
1 &               0.66 &                 0.66 \\
2 &               0.92 &                 0.88 \\
3 &               0.71 &                 0.70 \\
4 &               0.92 &                 0.88 \\
5 &               0.92 &                 0.88 \\
6 &               0.88 &                 0.82 \\
7 &               0.95 &                 0.88 \\
8 &               0.85 &                 0.90 \\
9 &               0.77 &                 0.72 \\
\bottomrule
\end{tabular}
\end{table}

